Created: July 12, 2023

Cleans and aggregates NCCS core data files on public charities for 2000-2019. Specific focus on health charities as categorized by NTEE major groups.

In [2]:
import pandas as pd
import os

# note: nccskey was dropped as an included col
COLS = ["EIN", "FISYR", "NAME", "STATE", "NTEE1", "ADDRESS", "CITY", "ZIP", "SUBSECCD", "NTMAJ5", "NTMAJ10", "NTMAJ12", "MAJGRPB", "CONT",
        "SOLICIT", "DIRSUPESTIMATE"]
DIR = '../data/working/'  

def clean(filename, year):
    path = os.path.join(DIR, filename)
    data = pd.read_csv(path, low_memory = False)
 
    # for consistency across column names
    data.columns = data.columns.str.upper()
    if 'CONT' not in data.columns.tolist():
        data = data.rename(columns = {'P1TCONT': 'CONT'})
    if 'SOLICIT' not in data.columns.tolist():
        data = data.rename(columns = {'NETINCFNDRSNG': 'SOLICIT'})

    # filter data
    df = data[COLS if ('DIRSUPESTIMATE' in data.columns.tolist()) else COLS[:-1]]
     
    # note: filtering out before 2000 and after 2019 since data is incomplete
    df = df[(df['CONT'] > 0) & (df['SUBSECCD'] == 3) & (df['NTMAJ5'] ==  "HE") & (df['FISYR'] < 2020) 
            & (df['FISYR'] > 1999)]
    
    return df  

/home/mee5vc/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
merged = None

for year in range(2000, 2020):   
    str_year = str(year)

    # only looking at public charities
    pc = clean('nccs.core' + str_year + 'pc.csv', year)
    
    # aggregate
    merged = pd.concat([merged, pc]).reset_index(drop = True)

merged = merged[merged['FISYR'] <= 2019]

# data set across all years
merged.to_csv("../data/distribution/merged_filtered.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../data/working/nccs.core2000pc.csv'